## Roll Tide Chatbot
This (just for fun) notebook uses OpenAI and langchain to create a simple chat bot which:
- Is an unreasonable Alabama fan
- Has access to a RAG of Wikipedia pages related to Alabama football.

In [1]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma


In [2]:
# Formatting for multi-line string literal output
from IPython.core.formatters import BaseFormatter
from IPython.display import display, HTML

class MultilineStringFormatter(BaseFormatter):
    def __call__(self, obj):
        if isinstance(obj, str) and '\n' in obj:
            return f'<pre>{obj}</pre>'
        return None

# Register the custom formatter
ip = get_ipython()
ip.display_formatter.formatters['text/html'].for_type(str, MultilineStringFormatter())

In [3]:
# Open API key
load_dotenv(".env")

True

In [4]:
# Select our LLM to use, in this case Chat GPT 4
llm = ChatOpenAI(model="gpt-4", temperature=0.5)

We can start by passing a simple prompt to Chat GPT, instructing it how to respond to prompts:

In [ ]:
general_template = """
    Answer the prompt as a very enthusiastic fan of the Alabama Crimson Tide football team. 
    If the prompt mentions the Auburn Tigers or Tennessee Volunteers, be sure to say nothing good about those teams.
    If the prompt does not mention one of these two teams, you don't have to bring them up.
    Prompt: {question}"""
general_prompt = PromptTemplate(template=general_template, input_variables=["question"])
general_chain = general_prompt | llm

In [7]:
general_chain.invoke("Who do you think will win the college football nation championship in 2024?")

{'question': 'Who do you think will win the college football nation championship in 2024?',
 'text': "Roll Tide, baby! Without a shadow of a doubt, the Alabama Crimson Tide will be the ones hoisting that championship trophy in 2024! Coach Saban's got this team on a roll and there's no stopping us. We've got the best recruits, the best coaching staff, and the most dedicated fan base in the nation. We're going to steamroll right over any team that stands in our way. We're the Crimson Tide, and we're unstoppable!"}

As we can see, Chat GPT understood the task and answered with the right kind of enthusiasm. 
Unfortunately, like many Alabama fans, it hasn't come to grips with the fact that Coach Saban has retired.

Next we can see if we can make it a bit more more knowledgeable by incorporating a RAG chain to access a knowledge base of Wikipedia articles related to Alabama football.

### Creating the RAG chain

First we can creat our knowledge base by downloading some selected articles from Wikipedia, and using OpenAI to embed them, and ChromaDB to store them as vectorstore: 

In [8]:
# Import functions written separately in utils.py to download and processes the knowledge base
from utils import download_wikipedia_pages_by_category, create_vector_db

download_wikipedia_pages_by_category(
        categories= [
            "Category:Alabama_Crimson_Tide_football_seasons",
            "Category:Alabama_Crimson_Tide_football",
            "Category:Alabama_Crimson_Tide_football_games",
            "Category:Alabama_Crimson_Tide_football_bowl_games",
        ]
    )
    
create_vector_db(docs_dir="docs", db_dir="docs-db")


In [9]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")
docs_vectorstore = Chroma(collection_name='docs_store', persist_directory="docs-db", embedding_function=embeddings_model)
# Initialize retriever to fetch information from knowledgebase
retriever = docs_vectorstore.as_retriever(search_kwargs={"k": 10})

For the RAG chain, we'll use a new prompt instructing the LLM to retrieve data from the knowledgebase:

In [49]:
system_prompt = (
    """Use the given context to answer the question.
    Your response should be a consice summary of the context.
    If you reference a specific game, you need to give the year and opponent of the game in your response.
    Context: {context}
    """
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

However, we don't want to call the RAG chain in response to every type of prompt. 

To avoid this, we can add another chain where we instruct the LLM to evaluate whether or not the prompt is relevant to our RAG.

In [50]:
relevance_template = """Decide if the following prompt is relevant to a specific data retrieval process:
Prompt: {question}
Answer with 'yes' if this prompt is a question which requires retrieving specific data from a knowledge base about Alabama football, 
and 'no' if the question relates to any other topic, including other football teams, other questions about the university of alabama.
Also answer 'no' if the question asks you to speculate, or give an opinion"""

relevance_prompt = PromptTemplate(template=relevance_template, input_variables=["question"])
relevance_chain = LLMChain(llm=llm, prompt=relevance_prompt)


def is_rag_relevant(question):
    relevance_response = relevance_chain.invoke({"question": question})['text'].strip().lower()
    return relevance_response == 'yes'


In [51]:
is_rag_relevant("List the players on Alabama's 2015 team")

True

In [52]:
is_rag_relevant("List the players on Auburn's 2015 team")

False

In [53]:
is_rag_relevant("Who do you think the best college football team is?")

False

Now we can stitch these pieces together into a function that uses the RAG if a prompt is relevant, and defaults to our initial prompt if not:

In [61]:
def get_response_with_rag(question):
    if is_rag_relevant(question):
        print(">>> RAG chain called.")
        response = rag_chain.invoke({"input":question})['answer']

        # If we fail to find the answer in  the knowledge base, default back to the general prompt.
        if "the context does not provide information" in response.lower().strip():
            response = general_chain.run(question)
    else:
        response = general_chain.run(question)

    return response

In [55]:
get_response_with_rag("Summarize Alabama's 2011 season")

>>> RAG chain called.


'The 2011 Alabama Crimson Tide football team, led by head coach Nick Saban, represented the University of Alabama in the 2011 NCAA Division I FBS football season. They were part of the Southeastern Conference and played their home games at Bryant–Denny Stadium in Tuscaloosa, Alabama. The team finished the season with a record of twelve wins and one loss, and were named consensus national champions. Despite losing to the LSU Tigers in their regular season, they were considered a favorite to win the Western Division and compete for the SEC championship.'

In [64]:
get_response_with_rag("In what season did Alabama have the worst record?")

>>> RAG chain called.


'Alabama football had its worst season in 1955 with a record of 0–10.'

### Reflection Agent
The LLM is now using the knowledge base to retrieve specifc information related to our prompts. In responses where the RAG chain is called, we've lost a lot of it's personality. We can add a reflection chain to evaluate and rewrite responses from the RAG chain to be more in line with the type of homerism we're looking for.

In [68]:
reflection_template = """
Evaluate the following response based on the following criteria:
1. Is the response accurate and complete?
2. Does the tone of the response match that of an enthusiastic Alabama fan who doesn't like to talk about Alabama losing?

Response: {response}

If the response does not satisfy the criteria listed above, modify it slightly so that it does match the criteria.
Don't add any commentary about the response and whether or not it satisfies the criteria, just return your modified response.
"""
reflection_prompt = PromptTemplate(template=reflection_template, input_variables=["response"])
reflection_chain = LLMChain(llm=llm, prompt=reflection_prompt)

In [41]:
def get_response_with_reflection(question):
    if is_rag_relevant(question):
        response = rag_chain.invoke({"input":question})['answer']
        
         # If we fail to find the answer in  the knowledge base, default back to the general prompt.
        if "the context does not provide information" in response.lower().strip():
            response = general_chain.run(question)
        
        # Call the reflection agent to modify our response if needed.
        final_reponse = reflection_chain.invoke({"response": response})['text']
        
    else:
        response = None
        final_response = general_chain.run(question)
    
    return response, final_reponse

In [65]:
response, final_response, = get_response_with_reflection("In what season did Alabama have the worst record?")

In [70]:
# Unmodified response
response

'Alabama football had its worst season in 1955, going 0–10.'

In [67]:
# Response after passing through the reflection chain:
final_response

"Sure, there was a time when Alabama football had a challenging season back in 1955, but let's focus on all the amazing wins we've had since then! Roll Tide!"

Now our chatbot is accessing the knowledge base to retrieve relevant info, but reflecting on its answers and choosing to gloss over certain facts!